## In this ntoebook we will generate data and plot to QC the stacks
### 1. Load data for 4-ROI stack from disk
### 2. Load 2-ROI stacks and ... 
####       a) generate 100 frame average array
####       b) write 50 pages of averaged stack to disk for rpeview
####       c) calculate mean and variance of each averaged plane for entire stack
### 3. Plot mean and variance of the new stack vs 4 ROI stack

In [ ]:
import meso_tools as mt
import os
import numpy as np
import matplotlib.pyplot as pl
import tifffile
import sciris as sc


### 1. Load data for 4 ROI stack - TBD


In [ ]:

path = "E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\4_roi_data.dict"
four_roi_data = sc.loadobj(path)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/nataliaorlova/Code/Packages/meso_tools/meso_tools/notebooks/E:\\dev_data\\OS_dendrites_stacks_pilots\\4roi_comparison_data\\4_roi_data.dict'


### 2. Load 2-ROI stacks



##### modify cell below to have correct path to the folder with 2 ROI stacks and corect names in teh roi_names list: in sequence they were acquire din stacks


In [ ]:

folder_with_2ROI_stacks = "E:\\dev_data\\OS_dendrites_stacks_pilots\\621602"
surface_soma_stack1_path = "E:\\dev_data\\OS_dendrites_stacks_pilots\\621602\\621602_V1_LM_surface_soma_00001.tif"
surface_soma_stack2_path = ""
roi_names= ['V1', 'LM', 'AM', 'PM']

In [ ]:

stacks = [surface_soma_stack1_path, surface_soma_stack2_path]
means = []
variances = []
rois=[]


In [ ]:

for stack in stacks:

    tiff=tifffile.TiffFile(stack, mode='rb')
    num_frames = len(tiff.pages)
    #stack parameters
    ROIs = 2
    image_dim = 768
    spacer = 24
    chunk_size = 100
    chunk_num = int(np.ceil(num_frames / chunk_size))
    print(f"Stack will be processed in {chunk_num} chunks")

    # Initiallize arrays for averaged stacks, mean and variance outputs
    roi1 = np.zeros((int(num_frames/chunk_size), image_dim , image_dim))
    roi2 = np.zeros((int(num_frames/chunk_size), image_dim , image_dim))

    mean1 = np.zeros((int(num_frames/chunk_size)))
    mean2 = np.zeros((int(num_frames/chunk_size)))

    var1 = np.zeros((int(num_frames/chunk_size)))
    var2 = np.zeros((int(num_frames/chunk_size)))


    # loop over chunks to generate averaged stack and calculate mean and variance

    for i in range(chunk_num):

        # load chunk of tiff:
        frame_start = chunk_size*i
        frame_end = chunk_size*(i+1)

        # handling of the last chunk 
        if frame_end > num_frames:
            frame_end = num_frames
        print(f'Loading {i}th chunk, frames {frame_start} to {frame_end}')
        tiff_array = tiff.asarray(range(frame_start,frame_end))

        num_frames_chunk = tiff_array.shape[0]

        # split and average 4 rois - single planes
        roi1[i,:,:] = tiff_array[:,:image_dim,:].mean(axis=0)
        roi2[i,:,:] = tiff_array[:,image_dim+spacer:2*image_dim+spacer,:].mean(axis=0)


        mean1[i] = np.mean(roi1[i,:,:],axis=(0,1))
        mean2[i] = np.mean(roi2[i,:,:],axis=(0,1))
        var1[i] = np.var(roi1[i,:,:])
        var2[i] = np.var(roi2[i,:,:])        

    tiff.close()

    means.append(mean1)
    means.append(mean2)
    variances.append(var1)
    variances.append(var2)
    rois.append(roi1)
    rois.append(roi2)


In [ ]:

# write stacks with averaged frames to disk
for i, roi_name in enumerate(roi_names):
    roi_path = os.path.join(folder_with_2ROI_stacks, f"{roi_name}_avg.tiff")
    mt.write_tiff(roi_path, rois[i].astype(np.int16))
    
    mean_path = os.path.join(folder_with_2ROI_stacks, f"{roi_name}_mean.dict")
    sc.saveobj(mean_path, means[i])
    
    var_path = os.path.join(folder_with_2ROI_stacks, f"{roi_name}_var.dict")
    sc.saveobj(var_path, variances[i])


In [ ]:

### 3. plot 2-roi vs 4-roi data

four_roi_mean = [four_roi_data['roi0_mean'], four_roi_data['roi1_mean'], four_roi_data['roi2_mean'], four_roi_data['roi3_mean']]
two_roi_mean = [means[0], means[1], means[2], means[3]]

four_roi_var = [four_roi_data['roi0_var'], four_roi_data['roi1_var'], four_roi_data['roi2_var'], four_roi_data['roi3_var']]
two_roi_var = [variances[0], variances[1], variances[2], variances[3]]


In [ ]:

four_roi_mean[0].shape

two_roi_mean[0].shape

fig, ax = pl.subplots(1,2, figsize=(16,5))

# plottnig 4-roi reference data
pl.sca(ax[0])
pl.ylabel('mean pixel value',fontsize=15)
pl.xlabel('plane',fontsize=15)
pl.title('4ROI Mean value per plane', fontsize=15)
print(f"Max intensity values for 4 ROI reference data: \n")
# pl.plot(np.array(four_roi_mean).T)
for i, mean in enumerate(four_roi_mean):
    line, = pl.plot(mean.T)
    line.set_label(roi_names[i])
    max_val = np.max(mean.T)
    print(f"Max value for {roi_names[i]} is {max_val}, at plane number {np.where(mean.T == max_val)[0][0]}")
pl.legend()

#plotting current stack data
pl.sca(ax[1])
pl.title('2ROI Mean value per plane', fontsize=15)  
print(f"\nMax intensity values for current stacks: \n")
for i, mean in enumerate(means):
    line, = pl.plot(mean)
    line.set_label(roi_names[i])
    max_val = np.max(mean.T)
    print(f"Max value for {roi_names[i]} is {max_val}, at plane number {np.where(mean.T == max_val)[0][0]}")
pl.ylabel('mean pixel value',fontsize=15)
pl.xlabel('plane',fontsize=15)
pl.legend()